## Genertaing Check Matrices Based on the 4-fold Cayley complex with PGL/PSL

In [30]:
# Loading the codes
import numpy as np
import pandas as pd 

# Loading the parameters: 
ma = 6
mb = 9 
degree = 14 
basedim = 120 
edgedim = degree * basedim
facedim = degree * edgedim

# reading the derived graph codes
# TGVEhor_idx = pd.read_csv('data/p13_q5/ramunujancsscodes_13_5_pgl_TGVEhor.csv').values.tolist()
# TGVEver_idx = pd.read_csv('data/p13_q5/ramunujancsscodes_13_5_pgl_TGVEver.csv').values.tolist()
# TGEFhor_idx = pd.read_csv('data/p13_q5/ramunujancsscodes_13_5_pgl_TGEFhor.csv').values.tolist()
# TGEFver_idx = pd.read_csv('data/p13_q5/ramunujancsscodes_13_5_pgl_TGEFver.csv').values.tolist()

# print(len(TGEFhor_idx))
# print(len(TGEFver_idx))
# print(len(TGVEhor_idx))
# print(len(TGVEver_idx))

# reading the X, Z parity check indices

idTGEF = pd.read_csv('data/p13_q5/ramunujancsscodes_13_5_pgl_TGEF.csv', header=None).values.tolist()
idTGVE = pd.read_csv('data/p13_q5/ramunujancsscodes_13_5_pgl_TGVE.csv', header=None).values.tolist()
print(len(idTGEF))
print(len(idTGVE))


168000
86400


In [31]:
from scipy import sparse
class BuildqcssCodes: 
    def __init__(self,ma:int, 
                mb:int, 
                delta:int, 
                basedim:int, 
                ) -> None:
        self.ma = ma
        self.mb =mb 
        self.delta = delta
        self.basedim = basedim 
        self.edgedim = delta * basedim
        self.facedim = delta * edgedim

    
    def build_codes(self, TGVEhor_idx:list, TGVEver_idx:list, TGEFhor_idx:list, TGEFver_idx:list):
        '''
        Build codes for the sparse index given in the matrix form
        (newest)
        '''
        TGEFhor = np.zeros((self.ma * self.edgedim ,self.facedim ), dtype=int)
        TGEFver = np.zeros((self.mb * self.edgedim ,self.facedim ), dtype=int)
        TGVEhor = np.zeros((self.ma * self.mb * self.basedim, self.ma * self.edgedim),dtype=int)
        TGVEver = np.zeros((self.ma * self.mb * self.basedim, self.mb * self.edgedim),dtype=int)
        for idx in TGEFhor_idx:
            idx = self._reindex(idx)
            # print(idx)
            # print(tuple(idx))
            TGEFhor[idx] = 1
        for idx in TGEFver_idx:
            idx = self._reindex(idx) 
            # print(idx)
            TGEFver[idx] = 1 
        for idx in TGVEhor_idx:
            idx = self._reindex(idx) 
            TGVEhor[idx] = 1 
        for idx in TGVEver_idx:
            idx = self._reindex(idx) 
            TGVEver[idx] = 1  
        self.TGEFhor = TGEFhor
        self.TGEFver = TGEFver
        self.TGVEhor = TGVEhor
        self.TGVEver = TGVEver
        TGEF = np.concatenate([self.TGEFhor, self.TGEFver], axis=0)
        TGVE = np.concatenate([self.TGVEhor, self.TGVEver], axis=1)
        return sparse.csr_matrix(TGEF), sparse.csr_matrix(TGVE)


    def build_chains(self, idTGVE:list, idTGEF:list): 
        
        '''
        Given derived graph code, we build explicit the X, Z parity checks. 
        '''

        TGEF = np.zeros((self.ma * self.edgedim + self.mb * self.edgedim, self.facedim), dtype=int)
        TGVE = np.zeros((self.ma * self.mb * self.basedim, self.ma * self.edgedim + self.mb * self.edgedim), dtype=int)
        for idx in idTGVE:
            idx = self._reindex(idx)
            TGVE[idx] = 1
        for idx in idTGEF:
            idx = self._reindex(idx)
            TGEF[idx] = 1 
        return TGVE, TGEF.transpose()

    def check_exactseq(self, TGEF:sparse.csr_matrix, TGVE: sparse.csr_matrix):
        '''
        check if XZ^T = 0 mod 2
        '''
        T = TGVE._mul_sparse_matrix(TGEF).toarray()
        # T = T.toarray()
        # np.where(T==1)
        # np.where(T>2)
        print(T[T % 2 ==1])
        assert len(T[T % 2 ==1]) == 0 # check if The CSS condition is fulfilled. 
    
    def check_exactseq2(self):
        '''
        check if XZ^T = 0 mod 2
        '''
        Thor = sparse.csr_matrix(self.TGVEhor)._mul_sparse_matrix(sparse.csr_matrix(self.TGEFhor)).toarray()
        Tver = sparse.csr_matrix(self.TGVEver)._mul_sparse_matrix(sparse.csr_matrix(self.TGEFver)).toarray()
        T = Thor + Tver 
        # T = T.toarray()
        # np.where(T==1)
        # np.where(T>2)
        print(len(T[T % 2 ==1]))
        assert len(T[T % 2 ==1]) == 0 # THe CSS condition is fulfilled.

    def check_lowdensity(self, Z: np.array, X: np.array):
        Z_row = np.count_nonzero(Z, axis=1)
        Z_col = np.count_nonzero(Z, axis=0)
        # print(len(Z_row))
        print(f'the maximum weight given Z stabilizer: {np.max(Z_row)}, should be bounded by {4 * max(self.ma, self.mb)} by Cayley Complexes ')
        print(f'the maximum number of Z stabilizers on a given qubit: {np.max(Z_col)}, should be bounded by {self.delta}')

        X_row = np.count_nonzero(X, axis=1)
        X_col = np.count_nonzero(X, axis=0)
        print(f'the maximum weight given X stabilizer: {np.max(X_row)}, should be bounded by {2 * self.delta}')
        print(f'the maximum number of X stabilizers on a given qubit: {np.max(X_col)}, should be bounded by {2 * max(self.ma, self.mb)}')
    
    def _reindex(self, index:list):
        new_index = []
        for i in index: 
            new_index.append(i -1)
        return tuple(new_index)


In [32]:
qCSS_codes = BuildqcssCodes(ma=6, mb=9, delta=14, basedim=120)
# .build_codes(TGVEhor_idx, TGVEver_idx, TGEFhor_idx, TGEFver_idx)
X, Z = qCSS_codes.build_chains(idTGVE, idTGEF)

In [33]:
print(X.shape) 
print(Z.shape)

(6480, 25200)
(23520, 25200)


#### Check Code Properties

In [35]:
# (1) Check for the CSS codes condition HxHz^T = 0

## checking if the CSS condition is sastfied
qCSS_codes.check_exactseq(sparse.csr_matrix(Z.transpose()), sparse.csr_matrix(X))

[]


In [36]:
# (2) check the low-density LDPC codes (apated from Lemma 3.2 from Vidick et al 2022)
qCSS_codes.check_lowdensity(Z, X)


the maximum weight given Z stabilizer: 10, should be bounded by 36 by Cayley Complexes 
the maximum number of Z stabilizers on a given qubit: 10, should be bounded by 14
the maximum weight given X stabilizer: 19, should be bounded by 28  
the maximum number of X stabilizers on a given qubit: 6, should be bounded by 18


In [37]:
# 3 Check the code rates adapted from the Lemma 3.3 from Vidick et al 2022 and thm 2.10.

def coderate(X, Z):
    X2 = np.shape(Z)[0]
    X1 = 2 * np.shape(Z)[1]
    X0 = 4 * np.shape(X)[0]
    return (X1 - X2 - X0) / X1

coderate(X, Z) # there is no surprise since we choose ma & mb =1 i.e too small.
#TODO: to change ma, mb such that the code rate would be positive. (This, however, comes with storage issue that I am fixing)

0.01904761904761905

In [38]:
print(np.linalg.matrix_rank(X))
print(np.linalg.matrix_rank(Z))

6480
